# REST APIs

https://developer.lightbend.com/guides/play-rest-api/part-1/index.html

**REST é uma forma de descrever como a arquitetura da Web - e outros sistemas semelhantes - é diferente.**

A característica principal que define REST são as **quatro regras de interface uniforme**. Por definição, REST APIs são APIs que seguem estas regras.

### Regra 1: Oferece acesso por meio de *recursos*

Em sistemas típicos, clientes e servidores trocam *comandos*. Suponha que queremos modelar uma lista em uma forma não-REST. Em uma linguagem técnica:

**NÃO-REST**
>*changeTodoList.php?item=35&action=changeTitle&title=new_title*

Note como isto é de fato uma instrução.

No estilo da arquitetura REST, servidores oferecem apenas recursos. Recursos são coisas conceituais sobre as quais clientes e servidores se comunicam.

**REST**
>*/todolists/7/items/35/*

Isto não é um comando, mas um endereço de um recurso. Você pode utilizar este endereço para manipular a lista utilizando operações padrão, em vez de comandos específicos da interface.

### Regra 2: Representa recursos por *representações*

Um recurso é uma "coisa" - e podemos descrevê-las em formatos diferentes. Por exemplo, humanos podem querer ver uma versão HTML, a qual seu browser transforma em um layout legível. Mas, às vezes, interfaces na Web são utilizadas por máquinas também. Elas precisam de um formato diferente, tal como *JSON*.

Em um formato não-REST, formatos diferentes possuem endereços diferentes:

**NÃO-REST**
>**browser**: */showTodoList.php?format=html

>**applicação**: */showTodoList.php?format=json

O problema é que sistemas que utilizam diferentes formatos não podem comunicar-se entre si, pois eles u tilizam endereços diferentes para a mesma coisa.

Em um sistema REST, endereços identificam coisas, não formatos, então todos os sistemas utilizam o mesmo endereço para a mesma coisa. Como eles identicam diferentes formatos? Eles pedem por eles explicitamente.

**REST**
>**browser**: "Eu quero */todolists/7/*, por favor me dê HTML."

>**aplicação**: "Eu quero *todolists/7/*, por favor me dê JSON".

A técnica que habilita isso é chamada *negociação de conteúdo*.

### Regra 3: Troca de mensagens auto-descritivas

Em um sistema REST, devemos ser capazes de interpretar qualquer mensagem sem haver visto a anterior. 

Imagine a seguinte conversa:

**NÃO-REST**
>1.*/search-results?q=todo*

>2.*/search-results?page=2*

>3.*/search-results?page=3*

O primeiro pedido recebe resultados de procura para "todo"; o segundo pedido recebe a segunda página disto. Agora, imagine que você quer ver apenas o segundo pedido. Como você sabe, como um servidor, o que fazer?

Em REST, cada pedido pode ser interpretada por si mesma.

**REST**
>1.*/search-results?q=to-do*

>2.*/search-results?q=todo&page=2*

>3.*/search-results?q=todo&page=3*

Outro aspecto disto, é que clientes e servidores REST usam apenas operações padrões, que são definidas em uma *especificação*. Para a Web, esta especificação é denominada *HTTP*.

### Regra 4: Conecta recursos por meio de *links*

Como se pode navegar em um website que você nunca viu anteriormente? Usa-se links! Você não precisa editar manualmente a barra de endereço no seu browser toda vez que você vai para uma nova página.

Em interfaces de máquina, isto não é sempre o caso. Suponha que uma aplicação pede por sua lista:

**NÃO-REST**
>*/todolists/7*

In [ ]:
{
    "name": "My-to-dos",
    "itens": [35,36]
}

Como você recebe os itens da lista? Você precisa ler a documentação.

Em REST, recursos conectam-se entre si por meio de hyperlinks:

**REST**
>*todolists/7/*

In [ ]:
{
    "name": "My to-dos"
    "items": ["/todolists/7/items/35/","/todolists/7/items/36"]
}

Note que não é necessário ler o manual para saber como recuperar os itens de sua lista. Você apenas segue os links.

# CRIANDO UMA REST API

Começaremos com uma REST API que mostra informação para posts de blog. Usuários devem ser capazes de digitar um título e um corpo de um post e criar novos posts, editar e deletar. 

### Modelando um Post Resource

A forma de fazer isto em REST é modelar o estado representado como um recurso. Um recurso de post de blog terá uma ID única, um hiperlink de URL que indica a localização canônica do recurso, o título de um post de um  blog, e o corpo.

Este recurso é representado como um único case class na aplicação Play:

In [1]:
case class PostResource(id: String, link: String,
                        title: String, body: String)

defined class PostResource

Este recurso é mapeado para e do JSON no front end usando Play, e é mapeado para e de um datastore persistente no backend utilizando um handler.

O Play manipula roteamento HTTP e representação para o REST API, e torna fácil escrever uma API assíncrona que é uma ordem de magnitude mais eficiente que outros frameworks para aplicações web.

### Encaminhamento de Post Requests

O Play possui dois mecanismos de roteamento complementares. No diretório *conf*, há um arquivo chamado "routes" o qual contém entradas para o método HTTP e um caminho URL relativo, e aponta para uma ação em um controller.

In [ ]:
GET              /               controllers.HomeController.index

Isto é útil para situações onde um serviço front end está renderizando HTML. No entanto, o Play contém um poderoso roteamento DSL que utilizaremos para a REST API.

Para qualquer request HTTP iniciando com */v1/posts*, o Play encaminha para uma classe dedicada *PostRouter* para manipular o recurso Posts, por meio do arquivo *conf/routes*:

In [ ]:
->            /v1/posts         v1.post.PostRouter

O *PostRouter* examina a URL e extrai dados para passar ao longo do controller: 

In [ ]:
package v1.post
import javax.inject.Inject

import play.api.mvc._
import play.api.routing.Router.Routes
import play.api.routing.SimpleRouter
import play.api.routing.sird._

class PostRouter @Inject()(controller: PostController)
  extends SimpleRouter
{
  override def routes: Routes = {
    case GET(p"/") =>
     controller.index
      
    case POST(p"/") =>
      controller.process

    case GET(p"/$id") =>
      controller.show(id)
  }
}

O roteamento DSL do Play (tecnicamente "String Interpolation Routing DSL, ou SIRD) mostra como dados podem ser extraídos da URL de forma limpa e concisa. O SIRD é baseado em métodos HTTP e um objecto extrator de string interpolada. Naturalmente, há também operadores para extrair queries, expressões regulares e adicionar extratores customizados. Se você possui uma URL:

In [ ]:
/posts/?sort=ascending&count=5

então você pode extrair os parâmetros "count" e "sort" em uma única linha:

In [ ]:
GET("/" ? q_?"sort=$sort" & q_?”count=${ int(count) }")

https://www.cakesolutions.net/teamblogs/all-you-need-to-know-about-plays-routing-dsl

### Usando um Controller

O PostRouter possui um PostController injetado nele por meio de injeção dependente padrão JSR-330:

In [ ]:
class PostRouter @Inject()(controller: PostController)
  extends SimpleRouter

Vamos discutir como controllers trabalham no Play.

Um controller lida com o trabalho de processar a requisição HTTP em uma resposta HTTP no contexto de uma ação: é onde uma página renderizando e processamento de formulário HTML acontecem. Um controller extende *play.api.mvc.Controller*, o qual contém um número de métodos utilitários e constantes para trabalhar com HTTP. Em particular, um Controller contém objetos de Resultado como Ok e Rediect, e nomes de cabeçalho como ACCEPT.

Os métodos em um controller consistem em um método retornando uma **Ação**. A ação provém o mecanismo para o Play.

Utilizando a ação, o controller passa um bloco de código que recebe uma **Requisição** passada como implícita - isto significa que qualquer método dentro do escopo que recebe uma requisição implícita como parâmetro utilizará essa requisição automaticamente. Então, o bloco deve retornar ou um **Resultado** ou um **Futuro[Resultado]**, dependendo se a ação foi chamada como *action {...}* ou *action.async{...}*.

### Lidando com GET Requests

Aqui está um exemplo simples de um Controller:

In [ ]:
import javax.inject.Inject
import play.api.mvc._

import scala.concurrent._

class MyController extends Controller {

  def index1: Action[AnyContent] = {
    Action { implicit request =>
      val r: Result = Ok("hello world")
      r
    }
  }

  def asyncIndex: Action[AnyContent] = {
    Action.async { implicit request =>
      val r: Future[Result] = Future.successful(Ok("hello world"))
      r
    }
  }
}

Neste exemplo, *index1* e *asyncIndex* possuem exatamente o mesmo comportamento. Internamente, não faz diferença se chamamos *Result* ou *Future[Result]*. No entanto, se você já está trabalhando com *Future*, async torna mais fácil 